# CS 484 :: Data Mining :: George Mason University :: Spring 2023


# Homework 3: Clustering&Association Rule Mining

- **100 points [9% of your final grade]**
- **Due Sunday, April 16 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; and (2) implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

## Part 1: Clustering (50 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by excuting the following code:

In [1]:
import numpy as np

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the model training process.**

In [2]:
import numpy as np
from sklearn.metrics import homogeneity_completeness_v_measure

# Load dataset
dataset = np.loadtxt("test.txt", delimiter=',')
features = dataset[:, 1:]
labels = dataset[:, 0]

class KMeans:
    
    def __init__(self, num_clusters=10, max_iterations=100, tolerance=0.0001):
        self.num_clusters = num_clusters
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        
    def fit(self, data):
        # Initialize centroids randomly
        self.centroids = data[np.random.choice(data.shape[0], self.num_clusters, replace=False)]        
        
        for iteration in range(self.max_iterations):
            # Assign each instance to the closest centroid
            distances = np.sqrt(((data - self.centroids[:, np.newaxis])**2).sum(axis=2))
            assigned_clusters = np.argmin(distances, axis=0)
            
            # Update centroids
            new_centroids = np.zeros((self.num_clusters, data.shape[1]))
            for cluster_index in range(self.num_clusters):
                new_centroids[cluster_index] = np.mean(data[assigned_clusters == cluster_index], axis=0)
            
            # Check if convergence criterion is met
            if np.all(np.sqrt(((new_centroids - self.centroids)**2).sum(axis=1)) < self.tolerance):
                break
                
            self.centroids = new_centroids

    def predict(self, data):
        distances = np.sqrt(((data - self.centroids[:, np.newaxis])**2).sum(axis=2))
        return np.argmin(distances, axis=0)
    

# Create and fit K-means model
kmeans_model = KMeans()
kmeans_model.fit(features)

Next, you need to calculate the square root of Sum of Squared Error (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged SSE of your algorithm.

In [3]:
# Calculate SSE for each cluster
assigned_clusters = kmeans_model.predict(features)
cluster_centroids = kmeans_model.centroids[assigned_clusters]
sse_per_cluster = np.sum((features - cluster_centroids)**2, axis=1)
sse = np.sum(sse_per_cluster)

# Calculate average SSE
average_sse = sse_per_cluster.mean()

# Print averaged SSE
print('Averaged SSE of K-means algorithm: ' + str(average_sse))

Averaged SSE of K-means algorithm: 2538658.051873758


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

In [4]:
# Calculate homogeneity, completeness, and v-measure
homogeneity_score, completeness_score, v_measure_score = homogeneity_completeness_v_measure(labels, kmeans_model.predict(features))

# Print the results
print('Homogeneity: ' + str(homogeneity_score))
print('Completeness: ' + str(completeness_score))
print('V-measure: ' + str(v_measure_score))

Homogeneity: 0.5080176516215134
Completeness: 0.5140663061668767
V-measure: 0.5110240810387741


## Part 2: Association Rule Mining (50 points)

In this part, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [5]:

user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id rated the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been rated by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user rating behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will highlight the method.**

In [6]:
import numpy as np
import pandas as pd
import itertools

# Load user-movie data from the file
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')

# Load movie names from the movies.csv file
movies_df = pd.read_csv("movies.csv")
movie_names = dict(zip(movies_df['movieId'], movies_df['movie_name']))

# Convert the data into a list of transactions
user_transactions = {}
# Iterate through each row in the user_movie_data
for row in user_movie_data:
    user_id = int(row[0])
    movie_id = int(row[1])
    # Create a set for each user to store the movies they have watched
    if user_id not in user_transactions:
        user_transactions[user_id] = set()
    # Add the movie to the user's set of watched movies
    user_transactions[user_id].add(movie_id)

# Convert the user_transactions dictionary to a list of transactions
transactions = list(user_transactions.values())

# Function to generate frequent single itemsets (1-itemsets)
def generate_frequent_single_itemsets(transactions, min_support):
    # Count the occurrences of each item in the transactions
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1

    # Identify the frequent 1-itemsets based on the min_support threshold
    frequent_single_itemsets = set()
    num_transactions = len(transactions)
    for item, count in item_counts.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_single_itemsets.add(frozenset([item]))

    # Print the number of candidates and frequent 1-itemsets
    print("Candidates of length 1 count: " + str(len(item_counts)))
    print("After Pruning count: " + str(len(frequent_single_itemsets)))

    return frequent_single_itemsets

# Function to generate frequent k-itemsets (k > 1)
def generate_frequent_itemsets(transactions, k, prev_frequent_itemsets, min_support):
    # Generate candidate k-itemsets by joining (k-1)-itemsets
    candidate_itemsets = set()
    prev_frequent_itemsets_list = list(prev_frequent_itemsets)
    for i in range(len(prev_frequent_itemsets_list)):
        for j in range(i + 1, len(prev_frequent_itemsets_list)):
            itemset1 = prev_frequent_itemsets_list[i]
            itemset2 = prev_frequent_itemsets_list[j]
            union = itemset1.union(itemset2)
            # Check if the union has k items
            if len(union) == k:
                itemset1_sorted = sorted(itemset1)
                itemset2_sorted = sorted(itemset2)
                # Check if the (k-1)-itemsets share the same (k-2) items
                if itemset1_sorted[:-1] == itemset2_sorted[:-1]:
                    # Check if all (k-1)-item subsets of the union are frequent
                    all_subsets_frequent = True
                    for subset in itertools.combinations(union, k - 1):
                        if frozenset(subset) not in prev_frequent_itemsets:
                            all_subsets_frequent = False
                            break
                    # If all (k-1)-item subsets are frequent, add the union to the candidates
                    if all_subsets_frequent:
                        candidate_itemsets.add(union)

    # Count the occurrences of candidate k-itemsets in the transactions
    item_counts = {}
    for transaction in transactions:
        for candidate_itemset in candidate_itemsets:
            # Check if the candidate k-itemset is a subset of the transaction
            if candidate_itemset.issubset(transaction):
                if candidate_itemset not in item_counts:
                    item_counts[candidate_itemset] = 0
                item_counts[candidate_itemset] += 1

    # Identify the frequent k-itemsets based on the min_support threshold
    frequent_itemsets = set()
    num_transactions = len(transactions)
    for itemset, count in item_counts.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_itemsets.add(itemset)

    # Print the number of candidates and frequent k-itemsets
    print("Candidates of length " + str(k) + " count: " + str(len(candidate_itemsets)))
    print("After Pruning count: " + str(len(frequent_itemsets)))

    return frequent_itemsets

# Function to generate association rules from the frequent itemsets
def generate_association_rules(transactions, frequent_itemsets, min_confidence):
    association_rules = []
    num_transactions = len(transactions)

    # Calculate the support of all frequent itemsets
    itemset_support = {}
    for transaction in transactions:
        for itemset in frequent_itemsets:
            if itemset.issubset(transaction):
                if itemset not in itemset_support:
                    itemset_support[itemset] = 0
                itemset_support[itemset] += 1

    # Calculate the support of all antecedents
    antecedent_support = {}
    for transaction in transactions:
        for itemset in frequent_itemsets:
            if len(itemset) > 1:
                # Create all possible antecedents of length i from the itemset
                for i in range(1, len(itemset)):
                    for antecedent in itertools.combinations(itemset, i):
                        antecedent = frozenset(antecedent)
                        if antecedent.issubset(transaction):
                            if antecedent not in antecedent_support:
                                antecedent_support[antecedent] = 0
                            antecedent_support[antecedent] += 1

    # Generate association rules from frequent itemsets
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in itertools.combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset.difference(antecedent)
                    antecedent_count = antecedent_support[antecedent]
                    itemset_count = itemset_support[itemset]
                    confidence = itemset_count / antecedent_count
                    # Check if the confidence is above the min_confidence threshold
                    if confidence >= min_confidence:
                        # Convert movie IDs to movie names
                        antecedent_names = [movie_names[movie_id] for movie_id in antecedent]
                        consequent_names = [movie_names[movie_id] for movie_id in consequent]
                        # Create a rule string and add it to the association_rules list
                        rule = ", ".join(antecedent_names) + " -> " + ", ".join(consequent_names)
                        association_rules.append(rule)

    return association_rules

# Parameters
min_support = 0.2
min_confidence = 0.8

# Generate frequent 1-itemsets
frequent_1_itemsets = generate_frequent_single_itemsets(transactions, min_support)

# Generate frequent itemsets of length k and association rules
frequent_itemsets = frequent_1_itemsets
k = 2
rules = []
while len(frequent_itemsets) > 0:
    prev_frequent_itemsets = frequent_itemsets
    # Generate frequent k-itemsets using the previous frequent itemsets
    frequent_itemsets = generate_frequent_itemsets(transactions, k, prev_frequent_itemsets, min_support)
    # Generate association rules from the current frequent itemsets
    association_rules = generate_association_rules(transactions, frequent_itemsets, min_confidence)
    # Add the generated association rules to the list of rules
    for rule in association_rules:
        rules.append(rule)
    # Increment k for the next iteration
    k += 1

# At this point, the 'rules' list contains the association rules that satisfy the given min_support and min_confidence thresholds.


Candidates of length 1 count: 408
After Pruning count: 21
Candidates of length 2 count: 210
After Pruning count: 36
Candidates of length 3 count: 55
After Pruning count: 12
Candidates of length 4 count: 1
After Pruning count: 0


Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [7]:
# print the association rules
for rule in rules:
    print(rule)
    
# Based on a piazza comment from the instructor these appear to be correct, but there are 5 rules missing.

Jaws (1975) -> Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993), Princess Bride, The (1987) -> Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993), Back to the Future (1985) -> Star Wars: Episode IV - A New Hope (1977)
Saving Private Ryan (1998), Back to the Future (1985) -> Star Wars: Episode IV - A New Hope (1977)
Back to the Future (1985), Schindler's List (1993) -> Star Wars: Episode IV - A New Hope (1977)
Saving Private Ryan (1998), Princess Bride, The (1987) -> Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993), Saving Private Ryan (1998) -> Star Wars: Episode IV - A New Hope (1977)
Godfather, The (1972), Godfather: Part II, The (1974) -> Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode IV - A New Hope (1977), Godfather: Part II, The (1974) -> Godfather, The (1972)
